# 第三方訂單記錄

In [ ]:
import pandas as pd
webside = pd.read_excel('/Users/daniel0522/Downloads/2022訂單報表.xlsx')
webside

In [ ]:
webside['訂單日期'] = pd.to_datetime(webside['訂單日期'])
webside['付款日期'] =  pd.to_datetime(webside['付款日期'])
webside.info()
webside.head()

In [ ]:
# 使用 loc 進行欄位修改
webside.loc[:, '訂單日期(日)'] = webside['訂單日期'].dt.date
webside.loc[:, '訂單日期(時間)'] = webside['訂單日期'].dt.time
# 嘗試確保日期正確解析
webside['付款日期'] = pd.to_datetime(webside['付款日期'], errors='coerce')
webside.loc[:, '訂單號碼'] = webside['訂單號碼'].str[1:]

webside.head()

In [ ]:
count_webside = len(webside['訂單號碼'].unique())
count_webside == len(webside['訂單號碼']),count_webside

In [ ]:
print('webside總共有',len(webside),'筆')
print('webside在2022有',len(webside[webside['付款日期'].dt.year == 2022]),'筆')
print('webside在2023有',len(webside[webside['付款日期'].dt.year == 2023]),'筆')

# 官網訂單紀錄

In [ ]:
GL = pd.read_excel('/Users/daniel0522/Downloads/2022 7-11 C2C全.xlsx',sheet_name = '代收款')
GL

In [ ]:
GL.info()
GL.head()
GL['訂單成立日'] = pd.to_datetime(GL['訂單成立日'])
GL['發貨日期'] = pd.to_datetime(GL['發貨日期'])
GL['取貨日期'] = pd.to_datetime(GL['取貨日期'])
GL['取貨日期'] = pd.to_datetime(GL['取貨日期'])

In [ ]:
count_GL = len(GL['訂單號碼'].unique())
count_GL == len(GL['訂單號碼']),count_GL

In [ ]:
duplicates = GL[GL['訂單號碼'].duplicated(keep = False)] # keep=False 會顯示所有重複記錄
duplicates

In [ ]:
dupicate_test = GL[GL['訂單號碼'] == '20220509055028200']
dupicate_test

In [ ]:
duplicates = GL[GL['訂單號碼'].duplicated(keep = 'first')] # keep=False 會顯示第一筆重複記錄
duplicates

In [ ]:
GL_drop_duplicate = GL.drop_duplicates(subset=['訂單號碼'])

In [ ]:
GL_drop_duplicate

In [ ]:
print('GL共有',len(GL),'筆')
print('GL不重複有',len(GL['訂單號碼'].unique()),'筆')
print('GL重複有',len(duplicates),'筆')

# 看未收訂單

In [ ]:
merge = webside.merge(GL_drop_duplicate, how = 'left', left_on = '訂單號碼',right_on = '訂單號碼')
merge.info()
merge.head()

In [ ]:
#在網站但不在收到的
gone = merge[merge['代收金額'].isna()]
gone.info()
gone.head()

In [ ]:
gone['付款日期'] = pd.to_datetime(gone['付款日期'])

In [ ]:
gone_paid_by_2023 = gone[gone['付款日期'].dt.year == 2023]
gone_paid_by_2022 = gone[gone['付款日期'].dt.year == 2022]

gone_paid_by_2022.info()
gone_paid_by_2022.head()

In [ ]:
print('在2022有',len(gone_paid_by_2022),'筆遺失訂單')
print('在2023有',len(gone_paid_by_2023),'筆遺失訂單')

## 換方法看

In [ ]:
#已付款
merge2 = webside.merge(GL, how = 'inner', left_on = '訂單號碼',right_on = '訂單號碼')
merge2.info()
merge2.head()

In [ ]:
len(merge2),len(webside),len(GL)

In [ ]:
#取除重複值
merge2_drop_duplicate = merge2.drop_duplicates(subset=['訂單號碼'])
len(merge2_drop_duplicate)

In [ ]:
merge2_2022 = merge2_drop_duplicate[merge2_drop_duplicate['付款日期'].dt.year == 2022]
merge2_2022.info()

In [ ]:
print('全部已付款有',len(merge2),'筆')
print('全部已付款去除重複有',len(merge2_drop_duplicate),'筆')
print('2022已付款有',len(merge2_drop_duplicate[merge2_drop_duplicate['付款日期'].dt.year == 2022]),'筆')
print('2023已付款有',len(merge2_drop_duplicate[merge2_drop_duplicate['付款日期'].dt.year == 2023]),'筆')


# 檢查金額不一致

In [ ]:
a = []
for i in range(len(merge2)):
    if merge2['訂單合計'][i] != merge2['代收金額'][i]:
        a.append(merge2['訂單號碼'][i])
        
check = merge2[merge2['訂單號碼'] == a[0]]
check